# **JANATA HACK INDEPENDENCE DAY 2020 ML HACKATHON**

Appliaed a Variety of approaches tried and I got a diverse variety of results.

Models Applied :-

<br>

---


1.   Deep Learning<br>
  *   One Layer CNN Model with GloVe ( 0.8029 ) and without GloVe ( 0.7973 ) 
  *   One Layer BiLSTM Model with GloVe ( 0.8083 ) without GloVe ( 0.7974 )
  *   One Layer BiGRU Model with GloVe ( 0.7924 )

  I could not get past this accuracy with Multiple trials and different hyperparameters. Saw an immense jump of models from around 75% to 80% with text cleaning (Helped convergence of loss of the training set and prevent overfitting) 

---

2.   Machine Learning
  *   Naive Bayes with CountVectorization ( 0.8028 )
  *   Logistic Regression with TFIDF( Word Level ) ( 0.7989 )
  *   Linear SVC with TFIDF
      *   With Squared_hinged loss ( 0.8065 ) 
      *   With hinged loss
          *    With loss_intercept=True ( 0.8174 )
          *    With loss_intercept=False ( 0.8190 )
               *    TFIDF( Char ) with n-gram (4-8) ( 0.8229 )

Somehow, I got accuracy a bit less when i cleaned the text separately. Although, TFIDF takes care of that. With breakthrough from hinged loss, I started trying different types of TFIDF and it turns out that the best approach was to use char level TFIDF with n gram. I tried varying different values of ngram range.




---



## Importing the Required Libraries

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

## Read and Assess the Dataset

In [ ]:
df_train=pd.read_csv("/content/train.csv")
df_test=pd.read_csv("/content/test.csv")

df_train.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0


## Extracting what we need

Here we have 2 columns that are Abstract and Title. We would need the content of both of these columns for our model to predict the topic. So, i created a new column in the dataframe called text, stored the test ids and deleted the columns ID, ABSTRACT,TITLE from the test and train dataframe

In [ ]:
df_train["text"]=df_train["TITLE"]+' '+df_train["ABSTRACT"]
df_test["text"]=df_test["TITLE"]+' '+df_test["ABSTRACT"]
del df_train["TITLE"]
del df_train["ABSTRACT"]
del df_train["ID"]
main_test_ids=df_test["ID"]
main_test_title=df_test["TITLE"]
main_test_abstract=df_test["ABSTRACT"]
del df_test["TITLE"]
del df_test["ABSTRACT"]
del df_test["ID"]

df_train.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,text
0,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps ...
1,1,0,0,0,0,0,Rotation Invariance Neural Network Rotation ...
2,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,0,0,1,0,0,0,A finite element approximation for the stochas...
4,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...


Extract the main columns by dropping the text. These will be used for validation.

In [ ]:
df_train["text"][1]
df_train_classes=df_train.drop("text",axis=1)
df_train_classes.head()

,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,1,0,0,1,0,0


## Optional Text Cleaning

In [ ]:
import re
def preprocess_text(sen):
    sentence = re.sub('[^a-zA-Z]', ' ', sen)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r"what's","what is",sentence)
    sentence = re.sub(r"won't","will not",sentence)
    sentence = re.sub(r" of ",' ',sentence)
    sentence = re.sub(r" and ",' ',sentence)
    sentence = re.sub(r" in ",' ',sentence)
    sentence = re.sub(r" this ",' ',sentence)
    sentence = re.sub(r" the ",' ',sentence)
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
X = []
sentences = list(df_train["text"])
for sen in sentences:
    X.append(preprocess_text(sen))
df_train["text"]=X
X=[]
sentences = list(df_test["text"])
for sen in sentences:
    X.append(preprocess_text(sen))
df_test["text"]=X

In [ ]:
df_train["text"][1]

'Rotation Invariance Neural Network Rotation invariance translation invariance have great values image recognition tasks In paper we bring new architecture convolutional neural network CNN named cyclic convolutional layer to achieve rotation invariance symbol recognition We can also get position orientation symbol by network to achieve detection purpose for multiple non overlap target Last but not least architecture can achieve one shot learning some cases using those invariance '

## Splitting into train and validation set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train["text"],df_train_classes, test_size=0.16, random_state=42)

## CountVectorizer used only for Naive Bayes only and not for other models

In [ ]:
cv = CountVectorizer(strip_accents="ascii", token_pattern=u"(?ui)\\b\\w*[a-z]+\\w*\\b", lowercase=True, stop_words="english")
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)
X_test_main_cv=cv.transform(df_test["text"])

## Training and Testing

I have done prediction label by label. made an array of all labels and predicted for each using SVC Model with char of n-gram

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

labels = ['Computer Science', 'Physics', 'Mathematics','Statistics','Quantitative Biology', 'Quantitative Finance']
label_predict=[]
for label in labels:
    text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=True,smooth_idf=False,sublinear_tf=True,analyzer='char',strip_accents='ascii', ngram_range=(4,8))),
                         ('clf',LinearSVC(loss="hinge",fit_intercept=False ,class_weight='balanced')),
    ])
# LinearSVC(loss="hinge",fit_intercept=False,tol=1e-3)
    text_clf.fit(X_train, y_train[label])  
# min_df=True,smooth_idf=True,sublinear_tf=True,analyzer='char',strip_accents='ascii',token_pattern=r'(?ui)\\b\\w*[a-z]+\\w*\\b', ngram_range=(4,8)
    predictions = text_clf.predict(X_test)

#     naive_bayes = MultinomialNB()
#     naive_bayes.fit(X_train_cv, y_train[label])
#     predictions = naive_bayes.predict(X_test_cv)
    
    print("Accuracy score: ", accuracy_score(y_test[label], predictions))
    print("Precision score: ", precision_score(y_test[label], predictions))
    print("Recall score: ", recall_score(y_test[label], predictions))
    
    final_predict= text_clf.predict(df_test["text"])
    print(final_predict)
    label_predict.append(np.array(final_predict))



Accuracy score:  0.8748510131108462
Precision score:  0.8146900269541779
Recall score:  0.8929098966026587
[0 0 1 ... 1 0 1]
Accuracy score:  0.9368295589988082
Precision score:  0.9121338912133892
Recall score:  0.872
[0 1 0 ... 0 0 0]
Accuracy score:  0.9055423122765197
Precision score:  0.8198689956331878
Recall score:  0.831672203765227
[0 0 0 ... 0 0 0]
Accuracy score:  0.8861740166865316
Precision score:  0.7593406593406593
Recall score:  0.8091334894613583
[1 0 0 ... 0 1 0]
Accuracy score:  0.9758641239570918
Precision score:  0.6904761904761905
Recall score:  0.29896907216494845
[0 0 0 ... 0 0 0]
Accuracy score:  0.9934445768772348
Precision score:  0.9
Recall score:  0.47368421052631576
[0 0 0 ... 0 0 0]


In [ ]:
label_predict.append(np.array(main_test_ids))
for x in label_predict:
    print(len(x))

8989
8989
8989
8989
8989
8989
8989


In [ ]:
dataset = pd.DataFrame({'ID':label_predict[6],'Computer Science': label_predict[0],'Physics': label_predict[1],'Mathematics': label_predict[2],'Statistics': label_predict[3],'Quantitative Biology': label_predict[4],'Quantitative Finance': label_predict[5]})

In [ ]:
dataset.head()

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,20973,0,0,0,1,0,0
1,20974,0,1,0,0,0,0
2,20975,1,0,0,0,0,0
3,20976,0,1,0,0,0,0
4,20977,1,0,0,0,0,0


In [ ]:
dataset.to_csv(r'submission.csv', index = False)